## Here is an example of how to check reverisibility of any matrix!

In [ ]:
import numpy as np

# P = matrix
# pi = linear equation system we have 

# Transition matrix
P = np.array([
    [0.3, 0.7, 0.0, 0.0],  # Downtown
    [0.2, 0.5, 0.3, 0.0],  # Suburbs
    [0.0, 0.0, 0.5, 0.5],  # Countryside
    [0.0, 0.0, 0.0, 1.0]   # Workshop
])


stationary_dist = stationary_distribution(P)


# Code for checking reversibility for ANY matrix!:

def is_reversible(P, stationary_dist, tol=1e-12):
    n = P.shape[0]
    for i in range(n):
        for j in range(n):
            left = stationary_dist[i] * P[i,j]
            right = stationary_dist[j] * P[j,i]
            if not np.isclose(left, right, atol=tol):
                return False
    return True

print("Reversible:", is_reversible(P, stationary_dist))




Reversible: True


## This is how you can always find the Stationary distribution of any matrix!

In [2]:
import numpy as np

def stationary_distribution(P, tol=1e-12, verify=True):
    """
    Compute the stationary distribution of a finite-state Markov chain.

    Parameters
    ----------
    P : np.ndarray, shape (n, n)
        Transition matrix. Rows should sum to 1 (row-stochastic).
    tol : float, optional
        Tolerance used for cleaning small numerical noise.
    verify : bool, optional
        If True, checks that the result is approximately stationary.

    Returns
    -------
    pi : np.ndarray, shape (n,)
        Stationary distribution vector (non-negative, sums to 1).

    Notes
    -----
    This solves the linear system:

        (P^T - I) * pi = 0,  with  sum(pi) = 1

    by replacing one of the equations with the normalization condition.
    It assumes that a (unique) stationary distribution exists.
    """
    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    if P.shape[0] != P.shape[1]:
        raise ValueError("P must be a square matrix.")

    # Build A * pi = b
    A = P.T - np.eye(n)
    b = np.zeros(n)

    # Replace last row with normalization condition: sum_i pi_i = 1
    A[-1, :] = 1.0
    b[-1] = 1.0

    # Solve linear system
    pi = np.linalg.solve(A, b)

    # Clean tiny numerical noise
    pi[np.abs(pi) < tol] = 0.0

    # If there are small negative values, clamp them to 0 and renormalize
    if np.any(pi < -tol):
        # Serious negativity -> indicate potential problem
        raise RuntimeError(
            "Computed stationary distribution has significantly negative entries. "
            "Check that P is a valid transition matrix with a unique stationary distribution."
        )

    # Clamp small negatives and renormalize
    pi = np.maximum(pi, 0.0)
    s = pi.sum()
    if not np.isfinite(s) or s <= 0:
        raise RuntimeError("Failed to compute a valid stationary distribution (sum <= 0).")
    pi /= s

    if verify:
        # Check stationarity: pi P ≈ pi
        if not np.allclose(pi @ P, pi, atol=1e-8):
            raise RuntimeError("Result does not satisfy pi P ≈ pi. Check the input matrix P.")

    return pi


In [ ]:
# This stationary distribution fiunction could also work, just not always the same and correct way:
def stationary_distribution(P):
    """
    Computes the stationary distribution of a Markov chain
    by finding the eigenvector corresponding to eigenvalue 1.
    """
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    
    # Find the eigenvector associated with eigenvalue 1
    idx = np.argmin(np.abs(eigenvalues - 1))
    vec = np.real(eigenvectors[:, idx])
    
    # Normalize to sum to 1
    stationary = vec / np.sum(vec)
    return stationary

### Always use this stationary distribution function below to check, it WILL always work! 

In [ ]:
def stationary_distribution_always_works(P):
    """
    Computes the stationary distribution by solving
    (P^T - I) * pi = 0  with  sum(pi) = 1.
    This method ALWAYS works for any Markov chain with a stationary distribution.
    """

    n = P.shape[0]

    # Build system: (P^T - I) * pi = 0
    A = P.T - np.eye(n)

    # Replace last equation with the normalization condition sum(pi)=1
    A[-1] = np.ones(n)

    b = np.zeros(n)
    b[-1] = 1.0

    # Solve the linear system
    pi = np.linalg.solve(A, b)

    return pi

### Expected hitting time function

This function computes expected hitting times to a given **target set of states** in a finite Markov chain with transition matrix \( P \).

We consider a Markov chain with state space \( \{0, 1, \dots, n-1\} \) and transition matrix

$$
P = (P_{ij})_{i,j=0}^{n-1},
$$

where \( P_{ij} = \mathbb{P}(X_{t+1} = j \mid X_t = i) \).

Given a set of **target states** \( T \subset \{0, \dots, n-1\} \), the *hitting time* of \( T \) is

$$
T_{\text{hit}} = \min\{ t \ge 0 : X_t \in T \}.
$$

For each state \( i \), we define the expected hitting time

$$
h(i) = \mathbb{E}[T_{\text{hit}} \mid X_0 = i].
$$

These satisfy

$$
h(i) = 0 \quad \text{for } i \in T,
$$

and for \( i \notin T \),

$$
h(i) = 1 + \sum_{j=0}^{n-1} P_{ij} h(j).
$$

If we collect the non-target states into a set \( S = \{0, \dots, n-1\} \setminus T \), and form the submatrix \( Q \) of \( P \) with rows and columns indexed by \( S \), then the vector \( h_S = (h(i))_{i \in S} \) solves

$$
(I - Q) h_S = \mathbf{1},
$$

where \( \mathbf{1} \) is a vector of ones.

The function `expected_hitting_time` implements this:

- **Parameters**
  - `P`: `np.ndarray` of shape `(n, n)`  
    Transition matrix of the Markov chain.
  - `target_states`: iterable of integers  
    Indices of the target states \( T \).
  - `start_state` (optional): integer  
    If provided, the function returns \( h(\text{start\_state}) \).
  - `start_dist` (optional): 1D array-like of length `n`  
    Initial distribution \( \alpha \). If provided, the function returns
    $$
    \mathbb{E}[T_{\text{hit}}] = \sum_{i=0}^{n-1} \alpha_i h(i).
    $$

- **Return value**
  - If `start_state` is given: a single float, \( h(\text{start\_state}) \).
  - If `start_dist` is given: a single float, the expected hitting time under that initial distribution.
  - If neither is given: a length-`n` NumPy array, containing \( h(i) \) for all states `i` (targets get value `0`).

- **Usage example (this exam problem)**

For the three-region chain

$$
P = \begin{pmatrix}
0.3 & 0.4 & 0.3 \\\\
0.2 & 0.5 & 0.3 \\\\
0.4 & 0.3 & 0.3
\end{pmatrix},
$$

with downtown = state 0 and suburbs = state 1:

```python
P = np.array([
    [0.3, 0.4, 0.3],  # Downtown
    [0.2, 0.5, 0.3],  # Suburbs
    [0.4, 0.3, 0.3],  # Countryside
])

ET_suburbs_to_downtown = expected_hitting_time(P, target_states=[0], start_state=1)
print(ET_suburbs_to_downtown)  # should be 50/13 ≈ 3.8461538


In [6]:

import numpy as np

def expected_hitting_time(P, target_states, start_state=None, start_dist=None):
    """
    Compute expected hitting times to a given set of target states in a finite Markov chain.

    Parameters
    ----------
    P : np.ndarray, shape (n, n)
        Transition matrix of the Markov chain.
    target_states : iterable of int
        Indices of the target states.
    start_state : int, optional
        If provided, return the expected hitting time starting from this state.
    start_dist : array-like, shape (n,), optional
        If provided, return the expected hitting time under this initial distribution.

    Returns
    -------
    float or np.ndarray
        - If start_state is given: expected hitting time from that state.
        - If start_dist is given: expected hitting time under that distribution.
        - If neither is given: array h of length n with expected hitting times
          from all states (targets have value 0).

    Notes
    -----
    This solves the linear system

        (I - Q) h_S = 1

    where Q is the submatrix of P restricted to non-target states,
    and 1 is a vector of ones. Assumes that the target set is hit
    with probability 1 from the relevant starting states.
    """
    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    target_states = np.array(sorted(set(target_states)), dtype=int)
    all_states = np.arange(n, dtype=int)

    # Non-target states S
    non_target_states = np.array([s for s in all_states if s not in target_states], dtype=int)

    # If all states are targets, hitting time is identically zero
    if non_target_states.size == 0:
        h = np.zeros(n, dtype=float)
        if start_state is not None:
            return float(h[start_state])
        if start_dist is not None:
            start_dist = np.asarray(start_dist, dtype=float)
            return float(start_dist @ h)
        return h

    # Build Q and solve (I - Q) h_S = 1
    Q = P[np.ix_(non_target_states, non_target_states)]
    I = np.eye(Q.shape[0])
    ones = np.ones(Q.shape[0])

    # Solve for h_S
    h_S = np.linalg.solve(I - Q, ones)

    # Put back into full vector h of length n
    h = np.zeros(n, dtype=float)
    h[target_states] = 0.0
    for idx, s in enumerate(non_target_states):
        h[s] = h_S[idx]

    # Return according to user request
    if (start_state is not None) and (start_dist is not None):
        raise ValueError("Provide either start_state or start_dist, not both.")

    if start_state is not None:
        return float(h[start_state])

    if start_dist is not None:
        start_dist = np.asarray(start_dist, dtype=float)
        if start_dist.shape[0] != n:
            raise ValueError("start_dist must have length equal to number of states.")
        return float(start_dist @ h)

    return h


In [7]:
# Example from above code: 

P = np.array([
    [0.3, 0.4, 0.3],  # Downtown
    [0.2, 0.5, 0.3],  # Suburbs
    [0.4, 0.3, 0.3],  # Countryside
])

# Expected steps until first time in Downtown (state 0) starting from Suburbs (state 1)
ET_suburbs_to_downtown = expected_hitting_time(P, target_states=[0], start_state=1)
print(ET_suburbs_to_downtown)  # ~3.846153846153846 (50/13)


3.846153846153846
